In [50]:
import pandas as pd

file_names = [
    '../data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv', 
    '../data/Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv', 
    '../data/Resale flat prices based on registration date from Jan-2017 onwards.csv'
]

for file in file_names:
    print("="*80)
    print(f"Reviewing file: {file}")
    try:
        df = pd.read_csv(file)
        print("Shape:", df.shape)
        
        print("\nColumns:")
        for i, col in enumerate(df.columns, start=1):
            print(f"{i}. {col} (dtype: {df[col].dtype})")
        
        print("\nFirst 5 rows:")
        print(df.head())
        
        print("\nDistinct value counts per column:")
        for col in df.columns:
            unique_count = df[col].nunique()
            print(f"{col}: {unique_count} unique values")
    except Exception as e:
        print("Error reading file:", file)
        print(e)
    print("\n")


Reviewing file: ../data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv
Shape: (287196, 10)

Columns:
1. month (dtype: object)
2. town (dtype: object)
3. flat_type (dtype: object)
4. block (dtype: object)
5. street_name (dtype: object)
6. storey_range (dtype: object)
7. floor_area_sqm (dtype: float64)
8. flat_model (dtype: object)
9. lease_commence_date (dtype: int64)
10. resale_price (dtype: int64)

First 5 rows:
     month        town flat_type block       street_name storey_range  \
0  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
1  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     04 TO 06   
2  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
3  1990-01  ANG MO KIO    1 ROOM   309  ANG MO KIO AVE 1     07 TO 09   
4  1990-01  ANG MO KIO    3 ROOM   216  ANG MO KIO AVE 1     04 TO 06   

   floor_area_sqm      flat_model  lease_commence_date  resale_price  
0            31.0        IMPROVED                 1977     

In [51]:
import pandas as pd
import numpy as np

file_names = [
    '../data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv', 
    '../data/Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv', 
    '../data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv', 
    '../data/Resale flat prices based on registration date from Jan-2017 onwards.csv'
]

initial_counts = []
final_counts = []
dfs = []

def parse_remaining_lease(lease_val):
    """
    Converts a lease value from a string like "86 years 1 months" to a numeric value (in years).
    Option 1 (default): returns years + (months/12).
    Option 2 (if you want to clamp months): simply return the years portion.
    """
    try:
        # If it's a string, split and parse
        if isinstance(lease_val, str):
            lease_val = lease_val.replace(',', '').strip()
            parts = lease_val.split()
            # parts[0] should be the number of years
            years = float(parts[0])
            # If there is a month component, process it:
            if len(parts) >= 3:
                months = float(parts[2])
                return years + months / 12.0
              
            else:
                return years
        else:
            return float(lease_val)
    except Exception as e:
        return np.nan

for file in file_names:
    df = pd.read_csv(file)
    
    initial_count = len(df)
    initial_counts.append(initial_count)
    
    df = df.dropna()
    cleaned_count = len(df)
    final_counts.append(cleaned_count)
    
    if 'remaining_lease' not in df.columns:
        # Compute remaining_lease for older files:
        df['year'] = df['month'].str[:4].astype(int)
        df['remaining_lease'] = (df['lease_commence_date'] + 99) - df['year']
        df.drop(columns=['year'], inplace=True)
    else:
        # Convert string formatted remaining_lease to numeric
        df['remaining_lease'] = df['remaining_lease'].apply(parse_remaining_lease)
    
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

total_initial = sum(initial_counts)
total_final = len(merged_df)
lost_rows = total_initial - total_final

print("Total initial rows across all tables:", total_initial)
print("Total rows after cleaning and computing remaining_lease:", total_final)
print("Total data points lost during cleaning:", lost_rows)



Total initial rows across all tables: 948962
Total rows after cleaning and computing remaining_lease: 948962
Total data points lost during cleaning: 0


In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


df = pd.read_csv("../data/merged_data.csv")


# These columns have few unique values so one-hot encoding is appropriate.
df = pd.get_dummies(df, columns=["town", "flat_type"], drop_first=True)

# The storey_range is a string like "10 TO 12". We extract the minimum and maximum floors,
# then compute their average.
df["min_floor"] = df["storey_range"].str[:2].astype(int)  # Extracts first two characters as min floor
df["max_floor"] = df["storey_range"].str[-2:].astype(int)  # Extracts last two characters as max floor
df["avg_floor"] = (df["min_floor"] + df["max_floor"]) / 2     # Computes the average floor
# Remove the original storey_range and the temporary min/max columns
df.drop(columns=["storey_range", "min_floor", "max_floor"], inplace=True)

# Target encoding replaces each category with the mean of the target variable (resale_price)
# for that category.
te = TargetEncoder(cols=["street_name", "block"])
# We transform these columns based on the overall resale_price means.
# (In a train/test setting, be sure to fit on the training set only to avoid data leakage.)
df[["street_name", "block"]] = te.fit_transform(df[["street_name", "block"]], df["resale_price"])

# Here we standardize floor_area_sqm, remaining_lease, and the computed avg_floor.
scaler = StandardScaler()
df[["floor_area_sqm", "remaining_lease", "avg_floor"]] = scaler.fit_transform(
    df[["floor_area_sqm", "remaining_lease", "avg_floor"]]
)

# The dataframe now has:
# - One-hot encoded columns for town and flat_type.
# - Target-encoded numeric values for street_name and block.
# - A numeric column (avg_floor) derived from storey_range.
# - Normalized continuous features.

print(df.head())

     month          block  street_name  floor_area_sqm      flat_model  \
0  1990-01  236708.523598   256110.925       -2.508324        IMPROVED   
1  1990-01  236708.523598   256110.925       -2.508324        IMPROVED   
2  1990-01  236708.523598   256110.925       -2.508324        IMPROVED   
3  1990-01  236708.523598   256110.925       -2.508324        IMPROVED   
4  1990-01  305892.089196   256110.925       -0.879459  NEW GENERATION   

   lease_commence_date  resale_price  remaining_lease  town_BEDOK  \
0                 1977        9000.0         0.466078           0   
1                 1977        6000.0         0.466078           0   
2                 1977        8000.0         0.466078           0   
3                 1977        6000.0         0.466078           0   
4                 1976       47200.0         0.374554           0   

   town_BISHAN  ...  town_WOODLANDS  town_YISHUN  flat_type_2 ROOM  \
0            0  ...               0            0                 0   


In [52]:
import pandas as pd

def analyze_numerical_data(df, title="Data Analysis"):
    numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
    print(f"=== {title} ===")
    print("Data Shape:", df.shape)
    print("\nNumeric Columns:", numeric_cols)
    print("\nSummary Statistics for Numeric Columns:")
    print(df[numeric_cols].describe())
    print("\nCorrelation Matrix for Numeric Columns:")
    print(df[numeric_cols].corr())
    print("\nUnique Value Counts for Numeric Columns:")
    for col in numeric_cols:
        print(f"{col}: {df[col].nunique()} unique values")
    print("\n" + "="*80 + "\n")

# --- Analysis for Raw Merged Data ---
raw_df = pd.read_csv("../data/merged_data.csv")
analyze_numerical_data(raw_df, "Raw Merged Data Analysis")

# --- Analysis for Processed Merged Data ---
proc_df = pd.read_csv("../data/processed_merged_data.csv")
analyze_numerical_data(proc_df, "Processed Merged Data Analysis")


=== Raw Merged Data Analysis ===
Data Shape: (948962, 11)

Numeric Columns: ['floor_area_sqm', 'lease_commence_date', 'resale_price', 'remaining_lease']

Summary Statistics for Numeric Columns:
       floor_area_sqm  lease_commence_date  resale_price  remaining_lease
count   948962.000000        948962.000000  9.489620e+05    948962.000000
mean        95.676691          1988.502663  3.278592e+05        80.907586
std         25.784833            10.890188  1.771736e+05        10.926094
min         28.000000          1966.000000  5.000000e+03        40.083333
25%         73.000000          1981.000000  1.950000e+05        74.000000
50%         93.000000          1986.000000  3.010000e+05        83.000000
75%        113.000000          1996.000000  4.250000e+05        90.000000
max        366.700000          2021.000000  1.600000e+06       101.000000

Correlation Matrix for Numeric Columns:
                     floor_area_sqm  lease_commence_date  resale_price  \
floor_area_sqm           